In [1]:
import pandas as pd
from pathlib import Path

In [2]:
folder_path = Path("c:/Users/AULA04/Desktop/Codes/zrive-ds/data/box_builder_dataset/feature_frame.csv")

In [3]:
feature_frame = pd.read_csv(folder_path)

In [11]:
for column in feature_frame:
    initial_checks_dataframe_column(feature_frame[column])

//0          33826472919172
1          33826472919172
2          33826472919172
3          33826472919172
4          33826472919172
                ...      
2880544    33826439594116
2880545    33826439594116
2880546    33826439594116
2880547    33826439594116
2880548    33826439594116
Name: variant_id, Length: 2880549, dtype: int64//
<class 'pandas.core.series.Series'>
RangeIndex: 2880549 entries, 0 to 2880548
Series name: variant_id
Non-Null Count    Dtype
--------------    -----
2880549 non-null  int64
dtypes: int64(1)
memory usage: 22.0 MB
None
/////
Total values: 2880549
Number of null values: 0 is the 0.00% of data
Duplicates Values: 2879573 is the 99.97% of data
Number of types: 1 types: [<class 'int'>]
The mean: 34012501571575.29
The standard deviation: 278624559685.6023
The min: 33615294398596 The max: 34543002157188 
Number of values uniques: 976
Error
//0             ricepastapulses
1             ricepastapulses
2             ricepastapulses
3             ricepastapulses
4 